In [1]:
import sys
import pandas as pd
import pymongo
import json
import os
import numpy

In [2]:
file = 'globalterrorismdb_0718dist.csv'
terror_df = pd.read_csv(file, encoding = "ISO-8859-1")

terror_df.head()

C:\Users\mjone\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
data = terror_df.groupby(['iyear', 'country_txt']).count()
terror_df = data.reset_index(level=['iyear', 'country_txt'])
df_index = terror_df.sort_values(by=['country_txt', 'iyear'])
table = df_index[['iyear','country_txt', 'eventid']].reset_index()

table['value_1'] = table.country_txt.shift()
answer=[]
total=0

for i in range(len(table)):
    if table['country_txt'][i] == table['value_1'][i]:
        newtotal = table['eventid'][i]+total
        total=newtotal
    else:
        newtotal=table['eventid'][i]
        total=table['eventid'][i]
    answer.append(newtotal)      

table['freq'] = answer
table =table[['iyear','country_txt','freq']].sort_values(by=['country_txt','iyear'])

frequency_df = pd.DataFrame({'name': table['country_txt'], 'time': table['iyear'], 'event': table['freq']})
frequency_df.head(100)

,name,time,event
0,Afghanistan,1973,1
1,Afghanistan,1979,4
2,Afghanistan,1987,5
3,Afghanistan,1988,16
4,Afghanistan,1989,26
5,Afghanistan,1990,28
6,Afghanistan,1991,58
7,Afghanistan,1992,94
8,Afghanistan,1994,103
9,Afghanistan,1995,109


In [5]:
column_names =list(terror_df)
column_names

['iyear',
 'country_txt',
 'eventid',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'region',
 'region_txt',
 'provstate',
 'city',
 'latitude',
 'longitude',
 'specificity',
 'vicinity',
 'location',
 'summary',
 'crit1',
 'crit2',
 'crit3',
 'doubtterr',
 'alternative',
 'alternative_txt',
 'multiple',
 'success',
 'suicide',
 'attacktype1',
 'attacktype1_txt',
 'attacktype2',
 'attacktype2_txt',
 'attacktype3',
 'attacktype3_txt',
 'targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
 'corp1',
 'target1',
 'natlty1',
 'natlty1_txt',
 'targtype2',
 'targtype2_txt',
 'targsubtype2',
 'targsubtype2_txt',
 'corp2',
 'target2',
 'natlty2',
 'natlty2_txt',
 'targtype3',
 'targtype3_txt',
 'targsubtype3',
 'targsubtype3_txt',
 'corp3',
 'target3',
 'natlty3',
 'natlty3_txt',
 'gname',
 'gsubname',
 'gname2',
 'gsubname2',
 'gname3',
 'gsubname3',
 'motive',
 'guncertain1',
 'guncertain2',
 'guncertain3',
 'individual',
 'nperps',
 'nperpcap',
 

In [6]:
dashboard_df = terror_df.loc[:, ["nkill","nwound","success","region","country_txt","iyear","region_txt",]]
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt
0,17,17,21,21,Argentina,1970,21
1,1,1,1,1,Australia,1970,1
2,1,1,1,1,Belgium,1970,1
3,1,1,1,1,Bolivia,1970,1
4,6,6,6,6,Brazil,1970,6


In [7]:
total_attacks = dashboard_df["success"].count()
total_attacks
total_wounded = dashboard_df["nwound"].count()
total_wounded

3762

In [8]:
total_fatalities = dashboard_df["nkill"].sum()
total_fatalities

171378

In [9]:
dashboard_df["total_fatalities"] = total_fatalities
dashboard_df["total_attacks"] = total_attacks
dashboard_df["total_wounded"] = total_wounded
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt,total_fatalities,total_attacks,total_wounded
0,17,17,21,21,Argentina,1970,21,171378,3762,3762
1,1,1,1,1,Australia,1970,1,171378,3762,3762
2,1,1,1,1,Belgium,1970,1,171378,3762,3762
3,1,1,1,1,Bolivia,1970,1,171378,3762,3762
4,6,6,6,6,Brazil,1970,6,171378,3762,3762


In [10]:
dashboard_df["region_txt"].unique()

array([  21,    1,    6,    2,   12,    3,    4,    5,    9,   10,  468,
         33,   20,    7,   11,   22,   35,   81,  247,    8,   13,   15,
         19,  292,   68,   24,   60,   14,  189,   58,   27,   71,   29,
         16,  203,   94,   28,   38,   39,  136,  194,  149,   54,  173,
         50,  105,   17,   80,   53,   47,  308,  147,  140,  130,   40,
         25,  158,   91,   59,   23,  277,   30,   76,   36,  209,   52,
        100,   87,   49,  326,  212,   82,   31,  206,   51,  279,  141,
        239,   69,   41,  710,  281,   67,   79,  110,   86,   64,  187,
         26,   95,  134,   56,   34,  172,  664,   66,  399,  108,   42,
        107,   18,  142,   74,  222,  537,   62,  354,  120,   63,  350,
        146,   78,  106,  234,  371,  121,  117,  299,  536,  122,  177,
         44,   46,  563,  237,  273,  145,  159,  166,  302,  592,   43,
        143,  232,  382,  436,  258,  352,  124,  114,  243,  307,  175,
         96,   48,  171,  568,  125,  337,  223,  6

In [11]:
success_failure = dashboard_df["success"].value_counts(1)

success_failure.head()

1    0.242690
2    0.111643
3    0.078947
4    0.052897
5    0.040404
Name: success, dtype: float64

In [12]:
region_values = dashboard_df["region_txt"].value_counts()
print(region_values)


1       913
2       420
3       297
4       199
5       152
6       117
8        93
7        83
11       61
9        59
10       56
12       52
13       45
15       37
17       35
14       34
19       31
16       30
18       30
20       28
21       27
23       24
22       20
35       20
29       19
27       19
24       18
33       16
26       15
41       15
       ... 
227       1
223       1
217       1
215       1
213       1
207       1
203       1
201       1
195       1
193       1
187       1
177       1
171       1
167       1
1308      1
165       1
163       1
135       1
129       1
119       1
101       1
99        1
93        1
85        1
1928      1
1469      1
1824      1
1654      1
1414      1
3933      1
Name: region_txt, Length: 360, dtype: int64


In [13]:
region_sum_df = pd.DataFrame(region_values)
region_sum_df.rename(columns={"region_txt":"total_terror"}, inplace=True)


In [14]:
incident_pyear = dashboard_df['iyear'].value_counts()
incident_pyear_df = pd.DataFrame(incident_pyear)

incident_pyear_df.head()

,iyear
1992,141
1994,136
1995,125
1996,123
1997,119


In [15]:
incident_pyear_df.rename(columns={"iyear":"incidents"}, inplace=True)
incident_pyear_df.head()

In [16]:
country_values = dashboard_df["country_txt"].value_counts()
country_values.mean()

18.351219512195122

In [17]:
country_bins = pd.DataFrame(country_values)
country_bins.rename(columns={"country_txt":"incidents"}, inplace=True)
country_bins.head()


,incidents
Lebanon,47
Greece,47
United Kingdom,47
United States,47
Spain,46


In [18]:
bins = [0,22,190, 24636]
group_names = ["Green", "Yellow", "Red"]
country_bins["danger_color"] = pd.cut(country_bins["incidents"], bins, labels=group_names)
country_bins.head()

,incidents,danger_color
Lebanon,47,Yellow
Greece,47,Yellow
United Kingdom,47,Yellow
United States,47,Yellow
Spain,46,Yellow


In [19]:
country_color = country_bins.groupby("danger_color")
country_color.head()

,incidents,danger_color
Lebanon,47,Yellow
Greece,47,Yellow
United Kingdom,47,Yellow
United States,47,Yellow
Spain,46,Yellow
Costa Rica,22,Green
Chad,22,Green
Panama,22,Green
Ukraine,21,Green
Democratic Republic of the Congo,21,Green


In [ ]:
# HERE IS A LIST OF ALL THE CLEANED AND READY DATA FRAMES
# dashboard_df = Most of the data for the dashboards. It is missing the bin information and the region totals of incidents
# country_danger_color = The bined amount of incidents per country and labeled green, yellow and red
# incidents_per_year = The amount of incidents that happened globally per year
# region_sum = Amount of incidents that has happened per region over all years
# total fatalities, total wounded, total attacks, success percent, and failure percent are all in the dashboard_df but I will list the values below
# total fatalities = 411868.0
# total attacks = 181691
# total wounded = 165380
# success percentage = 0.889598 or 89$
# failure percent = 0.110402 or 11%
#dashboard_df
#Country_danger_color
#incidents_per_year
#region_sum